Miguel Colom  
Rafael Grompone von Gioi  
Argyris Kalogeratos

TP3 : Optimisation Numérique  
Première Année Master Hadamard

# <center>Minimisation par optimisation proximale</center>

##  À savoir avant de commencer :
  * Vous pouvez préparer votre livrable en français ou en anglais.  
  
  * Justifiez de manière appropriée les solutions proposées dans le notebook en tant que réponses. Évitez d'ajouter un code qui n'est pas expliqué. Le notebook livré est la combinaison de code et de réponses bien élaborées.

  *  Prenez-garde à ce que votre code compile correctement et ne dépende pas d'autres modules que ceux indiqués dans le TP.

  *  Lorsque vous comparez plusieurs graphiques, veuillez utiliser la même figure. Ne créez pas de nouvelle figure pour chaque graphique.
  
  * Ajoutez chaque réponse juste en dessous de chaque question. Là, vous pouvez creer plusieur "cells" de code, de résultats/figures, ou de comentaires.

  *  $\rightarrow$ **Seuls les devoirs rendus sur eCampus sont acceptés!** Ils ne doivent jamais être envoyés par mail aux responsables des TP.
Veuillez vous assurer que votre accès à eCampus fonctionne correctement. Si ce n'est pas le cas, veuillez contacter immédiatement les responsables pédagogiques ou informatiques.

Remarque : la plupart du temps on n'implémente pas les méthodes classiques présentées dans ce TP et on utilise des paquets du type ``scipy``. Néanmoins, il convient de savoir ce que contienne ces _boîtes noires_ et de se forger une intuition sur les différentes méthodes d'optimisation.

Installer localement les paquets nécessaires:

In [ ]:
import sys
!{sys.executable} -m pip install --user scikit-learn
!{sys.executable} -m pip install --user scikit-image
!{sys.executable} -m pip install --user numpy

In [12]:
# Import basic packages
import numpy as np
import matplotlib.pyplot as plt

# Exercise 1. Implémentation de ISTA

L'algorithme ISTA [1] a pour objectif de minimiser une fonction $F : \mathbb{R}^d \to \mathbb{R}$ s'écrivant sous la forme $F = f + \lambda g$ où $\lambda >0$ est une constante,

1. $f$ est un fonction convexe, $C^1$, gradient Lipschitz i. e. pour tout $x,y \in \mathbb{R}^d$,

$$||\nabla f(x) - \nabla f(y)|| \leq L ||x-y||.$$

2. $g$ est une fonction continue convexe dont on peut calculer l'opérateur proximal $\text{prox}_g^{\gamma}$ défini pour tout $\gamma >0$ et $x \in \mathbb{R}^d$ par

$$\text{prox}_g^{\gamma}  = \text{argmin}_{y \in \mathbb{R}} \{g(y) + (2 \gamma)^{-1} ||x-y||^2 \}.$$

L'algorithme ISTA consiste alors à construire la suite $(x_k)_{k \in \mathbb{N}}$ partant de $x_0 \in \mathbb{R}^d$ à partir de la récursion suivante :

$$x_{k+1} = \text{prox}_{\lambda g}^{\gamma} \{x_k - \gamma \nabla f(x_k)\}.$$


## 1) Montrer que pour tout $\gamma, \lambda >0$, on a $$\text{prox}_{\lambda g}^{\gamma} = \text{prox}_{ g}^{\lambda \gamma}.$$

## 2) Recherche du minimum global

Implémenter alors l'algorithme ISTA. On pourra par exemple considérer un fonction sous la forme 

    ista(x0, prox_op_g, grad_f, fun_total, lambda_l, n_it=100):
        Variables d'entree :
            * x0 : solution initial
            * prox_op_g : operateur proximal de g qui prend en entree x et gamma
            * grad_f : fonction qui a partir d'un point x retourne le gradient de f et la constante de Lipschitz du gradient de f
            * fun_total : fonction F = f+lambda*g
            * lambda_l : parametre lambda dans F
            * n_it : nombre d'iterations

        Variables de sortie : x, fun_iterate
            * x : itere final de ISTA
            * fun_iterate : suite (F(x_k))

$\rightarrow$ **Plus précisément, nous nous référons à l'Algorithme 4 du Chapitre 7 du polycopié (pas fixe).**

# Exercise 2. Acquisition comprimée (_compressed sensing_)



Dans de nombreux problèmes, notamment en traitement du signal, seulement des observations partielles, regroupées dans un vecteur $y \in \mathbb{R}^p$, d'un signal $x \in \mathbb{R}^d$ sont disponibles. Cela s'explique par une volonté de compresser le signal par exemple ou encore parce que les instruments de mesures du signal sont dans certains cas peu précis. Mathématiquement, cette perte de données se modélise par une relation $y=Ax$, où $A \in \mathbb{R}^{p,d}$, et $p << d$. Comme $A$ n'est pas inversible, le plus souvent le problème inverse $y=Ax$ a de nombreuses solutions mais qui ne sont pas satisfaisantes. En effet, dans de nombreux cas, le signal d'origine $x$ est le plus souvent parcimonieux, i. e. un grand nombre de ses composantes sont nulles. Ainsi, pour reconstruire ce type de signal, il a été proposé de considérer le minimum de la fonction

$$F(x) = f(x) + \lambda g(x), \qquad f(x) = ||y-Ax||^2, \qquad g(x) =  \sum_{i=1}^d |x_i|,$$

où $\lambda >0$ est un coefficient qui contrôle le niveau de parcimonie de $x$.  On peut observer que $F$ peut s'écrire comme $f+g$ où $f$ est une fonction convexe gradient Lipschitz et $g$ est une fonction convexe continue mais non différentiable. On se propose dans cette partie d'appliquer l'algorithme ISTA à la minimisation de $F$
avec une application au traitement d'image. Ici $y$ sera une mesure d'une image et $x$ les coefficients en ondelettes de la même image. Ainsi la matrice $A$ est simplement l'opérateur linéaire qui à une représentation en ondelettes associe un signal.


## 1) Définir les fonctions suivantes :

In [7]:
def compute_wavelet_filter(type, par):
    """
        compute_wavelet_filter - Generate Orthonormal QMF Filter for Wavelet Transform


           [h,g] = compute_wavelet_filter(Type,Par)

         Inputs
           Type   string, 'Haar', 'Beylkin', 'Coiflet', 'Daubechies',
                  'Symmlet', 'Vaidyanathan','Battle'
           Par    integer, it is a parameter related to the support and vanishing
                  moments of the wavelets, explained below for each wavelet.

        Outputs
          h   low pass quadrature mirror filter
          g   high pass

         Description
           The Haar filter (which could be considered a Daubechies-2) was the
           first wavelet, though not called as such, and is discontinuous.

           The Beylkin filter places roots for the frequency response function
           close to the Nyquist frequency on the real axis.

           The Coiflet filters are designed to give both the mother and father
           wavelets 2*Par vanishing moments; here Par may be one of 1,2,3,4 or 5.

           The Daubechies filters are minimal phase filters that generate wavelets
           which have a minimal support for a given number of vanishing moments.
           They are indexed by their length, Par, which may be one of
           2,4,6,8,10,12,14,16,18 or 20. The number of vanishing moments is par/2.

           Symmlets are also wavelets within a minimum size support for a given
           number of vanishing moments, but they are as symmetrical as possible,
           as opposed to the Daubechies filters which are highly asymmetrical.
           They are indexed by Par, which specifies the number of vanishing
           moments and is equal to half the size of the support. It ranges
           from 4 to 10.

           The Vaidyanathan filter gives an exact reconstruction, but does not
           satisfy any moment condition.  The filter has been optimized for
           speech coding.

           The Battle-Lemarie filter generate spline orthogonal wavelet basis.
           The parameter Par gives the degree of the spline. The number of
           vanishing moments is Par+1.

        See Also
           FWT_PO, IWT_PO, FWT2_PO, IWT2_PO, WPAnalysis

        References
            The books by Daubechies and Wickerhauser.

        Warning : only Daubechies implemented for the moment !
    """

    if type == 'Daubechies':

        if par == 1:
            f = [1, 1] / np.sqrt(2)

        elif par == 4:
            f = [.482962913145, .836516303738,
                 .224143868042, -.129409522551]

        elif par == 6:
            f = [.332670552950, .806891509311,
                 .459877502118, -.135011020010,
                 -.085441273882, .035226291882]

        elif par == 8:
            f = [.230377813309, .714846570553,
                 .630880767930, -.027983769417,
                 -.187034811719, .030841381836,
                 .032883011667, -.010597401785]

        elif par == 10:
            f = [.160102397974, .603829269797, .724308528438,
                 .138428145901, -.242294887066, -.032244869585,
                 .077571493840, -.006241490213, -.012580751999,
                 .003335725285]

        elif par == 12:
            f = [.111540743350, .494623890398, .751133908021,
                 .315250351709, -.226264693965, -.129766867567,
                 .097501605587, .027522865530, -.031582039317,
                 .000553842201, .004777257511, -.001077301085]

        elif par == 14:
            f = [.077852054085, .396539319482, .729132090846,
                 .469782287405, -.143906003929, -.224036184994,
                 .071309219267, .080612609151, -.038029936935,
                 -.016574541631, .012550998556, .000429577973,
                 -.001801640704, .000353713800]

        elif par == 16:
            f = [.054415842243, .312871590914, .675630736297,
                 .585354683654, -.015829105256, -.284015542962,
                 .000472484574, .128747426620, -.017369301002,
                 -.044088253931, .013981027917, .008746094047,
                 -.004870352993, -.000391740373, .000675449406,
                 -.000117476784]

        elif par == 18:
            f = [.038077947364, .243834674613, .604823123690,
                 .657288078051, .133197385825, -.293273783279,
                 -.096840783223, .148540749338, .030725681479,
                 -.067632829061, .000250947115, .022361662124,
                 -.004723204758, -.004281503682, .001847646883,
                 .000230385764, -.000251963189, .000039347320]

        elif par == 20:
            f = [.026670057901, .188176800078, .527201188932,
                 .688459039454, .281172343661, -.249846424327,
                 -.195946274377, .127369340336, .093057364604,
                 -.071394147166, -.029457536822, .033212674059,
                 .003606553567, -.010733175483, .001395351747,
                 .001992405295, -.000685856695, -.000116466855,
                 .000093588670, -.000013264203]

        else:
            raise ValueError(f"Unimplemented for par={par}")

    else:
        raise ValueError("Wrong arguments, see comments for acceptable values")

    f = list(f / np.linalg.norm(f))

    if len(f) % 2 == 0:
        f = [0] + f

    return np.array(f)

In [10]:
from skimage import transform

def upsampling(x, d):
    """
    up-sampling along dimension d by factor p=2
    """
    p = 2
    s = x.shape
    if d == 1:
        y = np.zeros((p * s[0], s[1]))
        y[::p, :] = x
    elif d == 2:
        y = np.zeros((s[0], p * s[1]))
        y[:, ::p] = x
    else:
        raise NotImplementedError()
    return y


def subsampling(x, d):
    """
    subsampling along dimension d by factor p=2
    """
    p = 2
    if d == 1:
        y = x[::p, :]
    elif d == 2:
        y = x[:, ::p]
    else:
        raise NotImplementedError()
    return y


def rescale(f, a=0, b=1):
    """
    Rescale linearly the dynamic of a vector to fit within a range [a,b]
    """
    v = f.max() - f.min()
    g = (f - f.min()).copy()
    if v > 0:
        g = g * 1 / v
    return a * 1 + g * (b - a)


def load_image(name, n=-1, flatten=1, resc=1, grayscale=1):
    """
    Load an image from a file, rescale its dynamic to [0,1], turn it into a grayscale image
    and resize it to size n x n.
    """
    f = plt.imread(name)
    # turn into normalized grayscale image
    if grayscale == 1:
        if flatten == 1 and np.ndim(f) > 2:
            f = np.sum(f, axis=2)
    if resc == 1:
        f = rescale(f)
    # change the size of the image
    if n > 0:
        if np.ndim(f) == 2:
            f = transform.resize(f, [n, n], 1)
        elif np.ndim(f) == 3:
            f = transform.resize(f, [n, n, f.shape[2]], 1)
    return f


def circshift1d(x, k):
    """ 
    Circular shift of a 1D vector
    """
    return np.roll(x, -k, axis=0)


def cconv(x, h, d):
    """
    Circular convolution along dimension d.
    h should be small and with odd size
    """
    if d == 2:
        # apply to transposed matrix
        return np.transpose(cconv(np.transpose(x), h, 1))
    y = np.zeros(x.shape)
    p = len(h)
    pc = int(round(float((p - 1) / 2)))
    for i in range(0, p):
        y = y + h[i] * circshift1d(x, i - pc)
    return y


def reverse(x):
    """
    Reverse a vector. 
    """
    return x[::-1]


def perform_wavortho_transf(f, Jmin, dir, h):
    """
    perform_wavortho_transf - compute orthogonal wavelet transform

    fw = perform_wavortho_transf(f,Jmin,dir,options);

    You can give the filter in options.h.

    Works in 2D only.
    Copyright (c) 2014 Gabriel Peyre
    """

    n = f.shape[1]
    Jmax = int(np.log2(n) - 1)
    # compute g filter
    u = np.power(-np.ones(len(h) - 1), range(1, len(h)))
    # alternate +1/-1
    g = np.concatenate(([0], h[-1:0:-1] * u))

    if dir == 1:
        ### FORWARD ###
        fW = f.copy()
        for j in np.arange(Jmax, Jmin - 1, -1):
            A = fW[:2 ** (j + 1):, :2 ** (j + 1):]
            for d in np.arange(1, 3):
                Coarse = subsampling(cconv(A, h, d), d)
                Detail = subsampling(cconv(A, g, d), d)
                A = np.concatenate((Coarse, Detail), axis=d - 1)
            fW[:2 ** (j + 1):, :2 ** (j + 1):] = A
        return fW
    else:
        ### BACKWARD ###
        fW = f.copy()
        f1 = fW.copy()
        for j in np.arange(Jmin, Jmax + 1):
            A = f1[:2 ** (j + 1):, :2 ** (j + 1):]
            for d in np.arange(1, 3):
                if d == 1:
                    Coarse = A[:2**j:, :]
                    Detail = A[2**j: 2**(j + 1):, :]
                else:
                    Coarse = A[:, :2 ** j:]
                    Detail = A[:, 2 ** j:2 ** (j + 1):]
                Coarse = cconv(upsampling(Coarse, d), reverse(h), d)
                Detail = cconv(upsampling(Detail, d), reverse(g), d)
                A = Coarse + Detail
            f1[:2 ** (j + 1):, :2 ** (j + 1):] = A
        return f1


def noisy_observations(n=32, r_sparse=0.2, r_info=0.5):
    """
    Measurement function.

    Parameters:
    - n is the image size (n x n);
    - r_sparse is the ratio of non-zero coefficients (wavelet domain) of the
    signal x to recover;
    - r_info is the ratio between the size of y and the size of x.

    Return y, A,  where:
    - y is the vector of measurements;
    - A is the sensing matrix (we look for x such that y = Ax);
    """

    # Load Barbara's test imageimport urllib.requestimport urllib.request
    im = rescale(load_image("https://mcolom.perso.math.cnrs.fr/data/tps/optimisation/barb.png", n))

    h = compute_wavelet_filter("Daubechies", 4)

    # Compute the matrix of wavelet transform
    mask = np.zeros((n, n))
    A0 = []
    for i in range(n):
        for j in range(n):
            mask[i, j] = 1
            wt = perform_wavortho_transf(mask, 0, +1, h)
            A0.append(wt.ravel())
            mask[i, j] = 0
    A0 = np.asarray(A0)

    # Gaussian matrix x Wavelet transform (keep ratio r_info)
    G = np.random.randn(int(np.floor(n**2 * r_info)), n**2) / n
    A = G.dot(A0)

    # Threshold the image (keep ratio r_sparse) and generate the measurements y
    # Same as x_true = A0.T.dot(im.flatten())
    x_true = perform_wavortho_transf(im, 0, +1, h).ravel()
    thshol = np.sort(np.abs(x_true.ravel()))[int((1 - r_sparse) * n**2)]
    x_true[np.abs(x_true) <= thshol] = 0
    y = A.dot(x_true)  # Vector of measurements
    return y, A


def back_to_image(x):
    n = int(np.sqrt(x.size))
    h = compute_wavelet_filter("Daubechies", 4)
    wt = x.reshape((n, n))
    im = perform_wavortho_transf(wt, 0, -1, h)
    return im


def plot_image(x):
    plt.figure(figsize=(1, 1))
    im = back_to_image(x)
    plt.imshow(im, cmap='gray')
    plt.axis('off')


def total_variation_op(n=32):
    """
    Measurement function.

    Parameters:
    - n is the image size (n x n);

    Return T a total variation operator.
    """
    h = compute_wavelet_filter("Daubechies", 4)

    # Compute the matrix of wavelet transform
    mask = np.zeros((n, n))
    A0 = []
    for i in range(n):
        for j in range(n):
            mask[i, j] = 1
            wt = perform_wavortho_transf(mask, 0, +1, h)
            A0.append(wt.ravel())
            mask[i, j] = 0
    A0 = np.asarray(A0)

    # Total variation operator
    dx = np.eye(n**2)
    dx -= np.roll(dx, 1, axis=1)
    dx = np.delete(dx, np.s_[n - 1::n], axis=0)

    dy = np.eye(n**2)
    dy -= np.roll(dy, n, axis=1)
    dy = np.delete(dy, np.s_[-n:], axis=0)

    T = np.r_[dx, dy].dot(A0)  # TV in the image domain

    T = np.r_[np.eye(n**2), T]  # For usual L1 norm, add identity

    return T


def noisy_observation_inf(n=2**4):
    A = np.zeros((n, n))
    while np.linalg.det(A) == 0:
        A = np.random.randn(n, n)
    y = A.dot(np.ones(n)) + np.random.randn(n)
    return y, A


def noisy_observation_nuclear(n=2**4):
    A = np.random.binomial(1, 0.1, size=(n, n))
    while np.sum(A) <= n / 5:
        A = np.random.binomial(1, 0.1, size=(n, n))
    X = np.ones((n, n))
    Y = A * X
    return Y, A

In [13]:
import warnings
warnings.filterwarnings('ignore')

## 2) Générer les données $A$ et $y$ et fixer pour le moment $\lambda$

In [14]:
lambda_l1 = 1
y, A = noisy_observations()

## 3) Calculer le gradient de $f$ pour cet exemple

## 4) Dans cette question, on s'intéresse au calcul de l'opérateur proximal de $g$

### 4a) Soit $h$ une fonction convexe sci propre sur $\mathbb{R}^d$ sous la forme $h(x)= \sum_{i=1}^d h_i(x_i)$, pour tout $x = (x_i,\ldots,x_d)\in \mathbb{R}^d$, où pour tout $i \in \{1,\ldots,d\}$, $h_i : \mathbb{R} \to (-\infty, +\infty]$, est convexe sci propre. Montrer que pour tout $x \in \mathbb{R}^d$ et $\gamma >0$, $$\text{prox}_h^{\gamma}(x) = (\text{prox}_{h_i}^{\gamma}(x_i))_{i \in \{1, \ldots, d\}}.$$

### 4b) Considérons $\phi(t) = |t|$ pour tout $t \in \mathbb{R}$. Montrer que pour tout $u \in \mathbb{R}$ et $\gamma >0$ $$t^{\star} = \text{argmin}_{t \in \mathbb{R}} \{|t| + (2\gamma)^{-1} |t-u|^2\},$$ si et seulement si $$t^* \in u - \gamma \partial \phi(t^*).$$

### 4c) Déterminer la sous-différentielle de $\phi$ sur $\mathbb{R}$

### 4d) En distinguant les cas $|u| < \gamma$ et $|u| \geq \gamma$, déterminer pour tout $u \in \mathbb{R}$ et $\gamma >0$, $\text{prox}_{\phi}^{\gamma}(u)$

### 4e) En déduire $\text{prox}_{g}^{\gamma}(x)$ pour tout $x \in \mathbb{R}^d$ et $\gamma >0$

## 5) Implémenter le gradient de $f$ et l'opérateur proximal de $g$

## 6) Appliquer la fonction ista que vous avez précédemment codé pour minimiser $F$ et afficher l'image que vous obtenez à l'aide la fonction `plot_image`

## 7) Vérifier numériquement que l'ordre de convergence de ISTA est de l'ordre $O(k^{-1})$ où $k$ est le nombre d'itérations

## 8) Changer la valeur du paramètre $\lambda$ et afficher les images que vous obtenez. Discuter de vos résultats.

# Exercise 3. Complétion de matrice de faible rang

Dans cette section, on s'intéresse au problème de complétion de
matrice. Dans certains problèmes statistiques, on a accès à seulement
certaines entrées d'une matrice $X \in \mathbb{R}^{d \times d}$,
$Y = A \odot X$ où
$A =(\mathbf{1}_{I \times J}((i,j)))_{i,j \in \{1,\ldots,d\}}$, où
$I,J \subset \{1,\ldots,d\}$ et $\odot$ est la multiplication élément
par élément (ou appelé par certains "matlab").  Dans ce problème, on
voudrait alors retrouver les composantes manquantes de $X$. Pour cela,
il est commun de chercher une matrice $X$ de faible rang. Cela revient
alors à chercher à minimiser la fonction $F$ définie sur $\mathbb{R}^{d\times d}$ par
\begin{equation*}
  F(X) = f(X) + \lambda g(X), \qquad f(X) = ||Y-A\odot X||^2_{2}, \, g(X )= ||X||_{\star}  = \sum_{i=1}^d |\sigma_i(X)|,
\end{equation*}
où $||\cdot||_2$ est la norme de Frobenius et
$(\sigma_i)_{i \in \{1,\ldots,d\}}$ sont les valeurs singulières de
$X$. On rappelle le théorème de décomposition en valeurs propres
singulières:

__Théorème 1__
_Soit $A \in \mathbb{R}^{d \times m}$, $d \leq m$. Il existe alors deux matrices orthogonales $O_1,O_2$ et $\sigma_1(A) \geq \cdots \geq \sigma_d(A) \geq 0$ tel que $A = O_1 \Sigma O_2$ où $\Sigma_A \in \mathbb{R}^{d \times m}$ est la matrice dont les entrées sont données par $\Sigma_{i,i} = \sigma_i(A)$ pour $i \in\{1,\ldots,d\}$ et $\Sigma_{i,j} = 0$ sinon.  Les réels $(\sigma_1(A), \ldots, \sigma_d(A))$ sont appelés les valeurs singulières de $A$ et sont les valeurs propres de $A A^T$._


## 1) Montrer que $g$ est la norme dual de la norme opérateur $|||\cdot|||$ sur $\mathbb{R}^{d \times d}$, i. e.
  \begin{equation*}
    g(X) = \sup_{V \in \mathbb{R}^{d \times d}, \, |||V||| \leq 1} \langle X,V\rangle = \sup_{V \in \mathbb{R}^{d \times d}, \, |||V||| \leq 1}\text{Tr}(X^TV),
  \end{equation*}
 où $|||V|||$ est la norme spectral de $V$ (son valeur singulière le plus grand).

 En déduire que $g$ est convexe.
 

## On cherche alors à appliquer ISTA pour minimiser $F$

## 2) Générer les données $A$ et $y$ et fixer pour le moment $\lambda$

In [ ]:
lambda_nuclear = 1
Y, A = noisy_observation_nuclear()

## 3) Calculer le gradient de $f$ pour cet exemple

## 4) Dans cette question, on s'intéresse au calcul de l'opérateur proximal de $g$

### 4a) En utilisant que la norme de Frobenius est invariant par rotation, montrer que pour tout $X \in \mathbb{R}^{d \times d}$  le problème de minimisation
$$
  \min_{Y \in \mathbb{R}^{d\times d} } ||Y||_{\star} +(2\gamma)^{-1} ||Y-X||^2_2, \qquad\qquad\qquad (1)
$$
est équivalent à
$$
    \min_{D \in \mathsf{E}} ||D||_{\star} +(2\gamma)^{-1} ||D-\Sigma_X||^2_2, \qquad\qquad\qquad (2)
$$
où $\mathsf{E}$ est l'ensemble des matrices diagonales de $\mathbb{R}^{d \times d}$ et $\Sigma_X$ est la matrice diagonale donnée par la décomposition en valeurs singulières de $X$.  Pour cela on utilisera le résultat suivant [2].

__Théorème 2__
_Soit $A,B \in \mathbb{R}^{d \times d}$ deux matrices de dimension $d$. Alors pour toutes matrices orthogonales $O_1,O_2$,
$$
\langle O_1 A O_2,B\rangle \leq \langle\Sigma_A,\Sigma_B\rangle,
$$
où $\Sigma_A$ et $\Sigma_B$ sont les deux matrices diagonales données par la décomposition en valeurs singulières de respectivement $A$ et $B$._

Si $D$ est une solution de (2), comment construire $Y$ solution de (1) ?

### 4b) En déduire que pour tout $\gamma >0$ et $X \in \mathbb{R}^{d \times d}$
$$
      \text{prox}_g^{\gamma}(X) = U \text{prox}_{\ell_1}^{\gamma}(\Sigma_X) V,
$$
où $X = U \Sigma_X V$ est une décomposition en valeurs singulières
de $X$ et $\text{prox}_{\ell_1}^{\gamma}$ est l'operateur proximal de la
norme $\ell_1$, $Y \mapsto \sum_{i,j=1}^d |Y_{i,j}|$. 

## 5) Implémenter l'opérateur proximal de $g$

## 6) Tester l'algorithme ISTA et illustrer graphiquement sa convergence

# Références

[1] Amir Beck and Marc Teboulle. A fast iterative shrinkage-thresholding algorithm for
linear inverse problems. SIAM journal on imaging sciences, 2(1) :183–202, 2009.

[2] J. von Neumann, Some matrix-inequalities and metrization of matric-space,
Tomsk Univ. Rev, 1(11) :286--300, 1937.  